## Calculate drop chances and material sanity value

Values from [Penguin Statistics](https://penguin-stats.io/) can change between CN or US servers, CN used by default.

Assumptions which I don't know are correct:
- Don't include weekly stages tiers 1-4
- Pure gold can craft to 500 LMD
- Battle records can be crafted higher/lower based on their EXP value
- Coefficients are sum of drop rates for each item

Things which I might need to include?
- Moe devalues drills by 0.78368 of LS-5, how?
- Moe bases pure gold value based on factory production time, how?

In [1]:
import common as ak
import materials, craft, stage
import numpy as np
import numpy.typing as npt
from scipy.optimize import linprog

from drop_packet import DropPacket

%load_ext autoreload
%autoreload 2

### Get data from online/file

In [2]:
item_dict = ak.get_item_dict(lang="en_US", local=True)
stage_dict = ak.get_stage_dict(lang="zh_CN", local=True)
craft_dict = ak.get_craft_dict(lang="zh_CN", local=True)
psdf = ak.get_pengstats_df(server="CN")

Reading from local file
Reading from local file
Reading from local file


### Get name dictionaries
```
item_names: id -> name
item_names_rev: name -> id
```
Same for stages and events. `event_name` is eg "WR", "BI" etc.


In [3]:
stage_san_dict, stage_names, stage_names_rev, stage_lmd_dict = stage.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = stage.get_event_info(stage_names)

### Stages and materials to use to calculate MSV

Material IDs are from [materials.py](materials.py).
By default, they include materials, LMD, pure gold, skill summaries and battle records.

Stage IDs are main story, all permanent event stages and CE-5, LS-5 and CA-5.

In [4]:
material_ids = ak.get_material_ids(item_names_rev, materials.MATERIAL_NAMES)

In [5]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = stage.get_main_and_perm_stage_ids(psdf)
stage_ids_all = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

### Get crafting matrix involving all materials

In [6]:
craft_matrix, subprod_matrix = craft.get_craft_matrix(craft_dict, material_ids, item_names_rev)

### Get drops of all stages and package it up

Filter stages by removing all stages with either no drops or cost 0 sanity.
Apply a floor to the drop matrix, removing all drops lower than a certain percentage. By default, 1% 

In [7]:
drop_packet_all, _coeff_matrix_all = stage.get_drop_matrix(psdf, stage_ids_all, material_ids,
                                                          stage_lmd_dict, stage_san_dict)

drop_packet_all.filter_stages()

cutoff_threshold = 0.01
drop_packet_all.drop_matrix_cutoff(cutoff_threshold)

### Calculate MSV of all materials using the drop matrix prior

Minimise sanity value of all items, subject to constraints:
- In crafting, value of product is equal to sum of ingredients.
- Sanity return of a stage <= stage sanity cost

Coefficients are the sum of the drop chances for each item. Not sure if this is right, alternative way is with `coeff_matrix_all` which is 

$$
\frac{\text{sum of item loots on all stages}}{\text{sum of all stage clears which can drop that item}}
$$

Assume a crafting byproduct rate of 18% (because fuck you Nian, you never give me byproducts)

Use `scipy.optimize.linprog` to minimise the series of linear equations.

In [8]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix,
                                                         subprod_matrix, byprod_rate)

In [9]:
coeff = np.sum(drop_packet_all.drop_matrix, axis=0)
#coeff = _coeff_matrix_all

res = linprog(-coeff,
              A_ub=drop_packet_all.drop_matrix, b_ub=drop_packet_all.san_cost,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(drop_packet_all.item_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.4f}".format(item_names[d], msv[i]))

True
LMD: 0.0040
Pure Gold: 9.4197
Skill Summary - 1: 1.5655
Skill Summary - 2: 3.9801
Skill Summary - 3: 10.1190
Drill Battle Record: 0.7718
Frontline Battle Record: 1.5437
Tactical Battle Record: 3.8591
Strategic Battle Record: 7.7183
Orirock: 0.8592
Orirock Cube: 2.6497
Orirock Cluster: 13.0507
Orirock Concentration: 48.8105
Damaged Device: 3.0735
Device: 9.2926
Integrated Device: 36.9729
Optimized Device: 89.7897
Ester: 1.7987
Polyester: 5.4681
Polyester Pack: 21.6747
Polyester Lump: 90.1551
Sugar Substitute: 1.8899
Sugar: 5.7417
Sugar Pack: 22.7694
Sugar Lump: 96.0457
Oriron Shard: 2.2114
Oriron: 6.7062
Oriron Cluster: 26.6273
Oriron Block: 108.5097
Diketon: 2.2697
Polyketon: 6.8810
Aketon: 27.3266
Keton Colloid: 101.3022
Loxic Kohl: 22.8716
White Horse Kohl: 79.5131
Manganese Ore: 27.2721
Manganese Trihydrate: 95.6980
Grindstone: 30.1078
Grindstone Pentahydrate: 90.3155
RMA70-12: 37.2646
RMA70-24: 87.3002
Polymerization Preparation: 244.7147
Bipolar Nanoflake: 234.9081
D32 Steel:

## Toys, not related to calcing MSV

In [10]:
craft.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


### Calculate efficiency of event stages compared to main stages

$$
\text{stage efficiency} = \frac{\sum{\text{MSV} * \text{drop rate}}}{\text{stage cost}}
$$

If `>=1` then stage is as good or better to farm (yields more materials per sanity on average) than the best permanent stages.

In [11]:
event_stage_ids = stage.get_event_ids("WR", psdf, event_names_rev, remove_permanent=True)
event_drop_packet, _ = stage.get_drop_matrix(psdf, event_stage_ids, material_ids,
                                         stage_lmd_dict, stage_san_dict)
#print(event_drop_packet.drop_matrix)
stack_ = stage.print_stage_efficiency(event_drop_packet, msv, stage_names, item_names)
#for a, b, c in stack:
#    print("{}:\t\t{:.4}\t{}".format(stage_names[a], float(b), item_names[c]))
print(stack_)

          stage_id item_id efficiency     stage_name           item_name
0       act16d5_01   30021   0.566226           WR-1    Sugar Substitute
1   act16d5_01_rep   30021   0.597316   WR-1 (rerun)    Sugar Substitute
2       act16d5_02   30031   0.520228           WR-2               Ester
3   act16d5_02_rep   30031   0.544929   WR-2 (rerun)               Ester
4       act16d5_03   30041   0.481202           WR-3        Oriron Shard
5   act16d5_03_rep   30041   0.505486   WR-3 (rerun)        Oriron Shard
6       act16d5_04   30032   0.745523           WR-4           Polyester
7   act16d5_04_rep   30032   0.752249   WR-4 (rerun)           Polyester
8       act16d5_05   30052   0.847824           WR-5           Polyketon
9   act16d5_05_rep   30052   0.774993   WR-5 (rerun)           Polyketon
10      act16d5_06   30012    0.58689           WR-6        Orirock Cube
11  act16d5_06_rep   30012   0.584159   WR-6 (rerun)        Orirock Cube
12      act16d5_07   30042   0.823083           WR-

In [12]:
_ = stage.print_stage_drops(stage_names_rev["R8-11"], drop_packet_all, item_names)

LMD: 252
Crystalline Component: 52.79%
Orirock Cube: 36.70%
Orirock: 21.41%
Device: 13.95%
Pure Gold: 10.17%
Damaged Device: 7.96%
Orirock Cluster: 3.26%
Crystalline Circuit: 2.70%
Loxic Kohl: 2.66%
Coagulating Gel: 2.08%
Integrated Device: 1.81%


In [13]:
stages_ = stage.get_stages_which_drop("Compound Cutting Fluid", drop_packet_all.drop_matrix,
                                      material_ids, stage_ids_all, item_names_rev)
for s, d in stages_:
        print("{}: {:.4f}%".format(stage_names[s], 100*d))


10-17: 57.4899%
SN-1 (permanent): 27.2727%
9-6: 27.1717%
10-15: 1.8682%
10-14: 1.2518%
7-14: 1.2308%
4-8: 1.1294%
4-5: 1.1245%
9-2: 1.1129%
9-11: 1.0638%
6-4: 1.0582%
9-12: 1.0204%
9-19: 1.0083%


In [14]:
def get_expected_sanity(drop_packet: DropPacket, item_id: str, stage_id: str, n_items: int) -> float:
    espd = drop_packet.get_espd(item_id, stage_id)
    total_sanity = espd * n_items
    return total_sanity
    
def get_outstanding_op(drop_packet: DropPacket, item_id: str, stage_id: str, n_items: int,
                       days_left: int, daily_san = ak.DAILY_SAN, san_per_op = ak.SAN_PER_OP) -> float:
    total_san = get_expected_sanity(drop_packet, item_id, stage_id, n_items)
    outstanding_san = total_san - days_left * daily_san
    outstanding_op = outstanding_san/san_per_op
    return outstanding_op

In [15]:
remaining_op = get_outstanding_op(event_drop_packet, item_names_rev["Incandescent Alloy"],
                          stage_names_rev["WR-8 (rerun)"], 141, 7)
print("Remaining OP: {:.2f}".format(remaining_op))

Remaining OP: 11.18
